In [1]:
import DrWatson
@quickactivate "haram"

LoadError: LoadError: UndefVarError: @quickactivate not defined
in expression starting at In[1]:2

In [2]:
using main
using Plots, Random, Distributions, ProgressMeter
using Flux, DynamicPPL, Zygote

gr()
theme(:default)
default(fmt=:png, levels=7, lw=0.5, msw=0.5, la=0.5)
ProgressMeter.ijulia_behavior(:clear);

KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found